# Smoothly Broken Power Law

This program shows you how to calcualte the integrated photon and energy fluxes of the Smoothly Broken Power Law using two different methods.

1.  Integrating the Function in Wolfram Alpha and then passing an energy array to the integrated function.
2.  Using scipy.integrate to numerically integrate the function. This way is more precise.

### The equation for the Smoothly Broken Power Law can be found in the following paper:
### F. Ryde 1998, Equation 2.
http://arxiv.org/abs/astro-ph/9811462v1

### My version of this equation:
I use different parameters to represent the variables that can be used within python to run calculations.

In [1]:
%%latex
Original Function, before integration: 
\begin{equation}
f\left(E\right) = N \times \left(\frac{E}{100}\right)^{p2} \times  \left[ \frac{\cosh \left( \frac{\log_{10}(E/k)}{d} \right)}{\cosh \left( \frac{\log_{10}(100/k)}{d} \right)} \right] ^{(p1*d*\log(10))}  dE \ \ \ \ \ \ \ \         
\end{equation}

<IPython.core.display.Latex object>

Original Function, before integration: 
    
    alpha: low energy index
    beta:  high energy index
    d:     break scale
    k:     break energy (Ebreak)
    N:     normalization
    E:     energy to integrate over, 10 to 10,000 keV

    p1 = (beta - alpha)/2.
    p2 = (alpha + beta)/2.
    d  = 0.3           

    f(E) = 
        N * ((E/100)**p2 * ((cosh(log10(E/k)/d) / 
            cosh(log10(100/k)/d))**(p1*d*log(10)))) dE

In [2]:
%%latex
\begin{equation}
\cosh(x) = \frac{e^{x} + e^{-x}}{2}
\end{equation}

<IPython.core.display.Latex object>

# Begin Program

In [3]:
from __future__ import division
import numpy as np
from scipy import integrate

### Constants

In [4]:
keVtoerg    = 1.60217657E-9
emin        = 10.0
emax        = 10000.0

### Parameters
    pars = [alpha, beta, ebreak, norm]

In [5]:
pars     = [-1.1128469, -2.2118149, 274.8992, 0.014636453]  

# normalization
norm   = pars[-1]

def get_parVals():
    pars     = [-1.1128469, -2.2118149, 274.8992, 0.014636453]  
    return pars


# FUNCTION INTEGRATED

Make a flux array of 5000 elements that will be filled with flux increments.
An energy array with 5000 energies ranging from 10 to 10,000 keV in log space will be passed to the integrated SBPL function and will calculate an increment of flux for each energy element passed to it.  
Those increments will be stored in the flux array of 5000 elements.

After running sbpl(engs, flux, *pars) the fluxes (un-normalized) will be appended to the flux array.  Then you can multiply each flux element by the normalization factor.  
You could pass the normalization to the sbpl function and multiply it within the function instead of at the end.  
I do not do that here because the equation (shown below) was the exact one  used during PYXSPEC fitting.  The only difference between this function and the PYXSPEC one is that the order the variables passed to the sbpl function in PYXSPEC are:  sbpl(engs, params, flux)


If you sum up the 5000 increments in the flux array and then multiply by the the normalization, you are getting the integrated flux from 10 to 10,000 keV.
You are essentially summing up the flux area under the flux curve.


In this example, we leave the SBPL normalization parameter out of the function and multiply it at the end.  We do this because PYXSPEC uses this exact function for sbpl (as below) during the Maximum Likelihood fitting process of parameter estimation.  

This is how PYXSPEC estiamtes its flux calculations when you use the commands:

    AllModels.setEnergies("10. 10000.")
    AllModels.calcFlux("10. 10000.0 err")
 
This is a crude method to estimate the flux and is not the  most accurate way.  The best way to calculate the integrated flux is to take the original function and numerically integrate it with scipy.integrate.quad for quadrature integration.

    Photon Flux Units:  photons s^-1 cm^-2
    Energy Flux Units:  ergs s^-1 cm^-2




In [6]:
def sbpl(engs, flux, *params):
    from scipy.special import hyp2f1
    from numpy import log, log10, exp, cosh, sinh
    a  = float(params[0])  # alpha 
    b  = float(params[1])  # beta
    k  = float(params[2])  # ebreak
    d  = 0.3               # break scale.  delta

    for i in range(len(engs)-1):
        p1 = (b - a)/2.  # (beta - alpha)/2.
        p2 = (a + b)/2.  # (alpha + beta)/2.
        # LOG AND LOG10 CONSTANTS TO SIMPLIFY THE FUNCTION
        c1 = 4.60517     # log(100)
        c2 = 0.868589    # 2*(1./log(10)) see also logarithmic properties of log((1+n)/(1-n))
        c3 = 2.30259     # log(10)
        c4 = 0.434294    # 1./log(10)
        c5 = 1.59603     # log(2)*log(10) = log(2)+log(10)
        c6 = 1.15129     # log(10)/2.
 
        multiplier = ( (1./(b+1.)) * -exp(-c1*p2) * (cosh(c5*p1*d)-sinh(c5*p1*d)))

        lowIntegral = (
            engs[i]**(p2+1)
            * ( ((engs[i]/k)**(-c2/d) +1.)**(-c3*p1*d))
            * ((((engs[i]/k)**(-c4/d)) + ((engs[i]/k)**(c4/d)))**(c3*p1*d))
            * float(hyp2f1(-c3*p1*d, -c6*d*(b+1.), -c6*d*(b+1.)+1., -(engs[i]/k)**(-c2/d)))
            * (cosh((c4 * log(engs[i]/k))/d)**(-c3*p1*d))
            * (((1./cosh((c4*(log(1/k)+c1))/d)) * cosh((c4*log(engs[i]/k))/d))**(c3*p1*d))
            )

        highIntegral = (
            engs[i+1]**(p2+1)
            * ( ((engs[i+1]/k)**(-c2/d) +1.)**(-c3*p1*d))
            * ((((engs[i+1]/k)**(-c4/d)) + ((engs[i+1]/k)**(c4/d)))**(c3*p1*d))
            * float(hyp2f1(-c3*p1*d, -c6*d*(b+1.), -c6*d*(b+1.)+1., -(engs[i+1]/k)**(-c2/d)))
            * (cosh((c4 * log(engs[i+1]/k))/d)**(-c3*p1*d))
            * (((1./cosh((c4*(log(1/k)+c1))/d)) * cosh((c4*log(engs[i+1]/k))/d))**(c3*p1*d))
            )

        val         = multiplier * (lowIntegral - highIntegral)
        flux[i]     = val
        

In [7]:
N      = 5000
engs   = np.logspace(1, 4, N)
flux   = np.zeros(N)

# WILL STORE CALCULATIONS IN FLUX ARRAY.
sbpl(engs, flux, *pars)
flux_ph = np.sum(flux) * norm

# NO NEED TO MAKE AN ESBPL FUNCTION.  MULTIPLY ENGS BY THE FLUX.
flux_en = np.sum(flux * engs * keVtoerg) * norm

print(
'''
Photon Flux:  %.9f \t photons s^-1 cm^-2
Energy Flux:  %.9e \t ergs s^-1 cm^-2
'''%(flux_ph, flux_en))


Photon Flux:  6.206339480 	 photons s^-1 cm^-2
Energy Flux:  2.002501937e-06 	 ergs s^-1 cm^-2



## FUNCTION WITHOUT BEING INTEGRATED.
### USING COSH

In [8]:
def sbpl(energy):
    from numpy import log, log10, cosh
    alpha, beta, ebreak, norm = get_parVals()
    eng  = energy
    k    = float(ebreak)
    N    = float(norm) 
    d    = 0.3  
    p1   = (beta - alpha)/2.
    p2   = (alpha + beta)/2.
    p3   = ( log10(100.0/k)/d )
    p4   = ( log10(eng/k)/d )
    return N * (((eng/100.)**p2) * ((cosh(p4) / cosh(p3)) **(p1*d*log(10.))) ) 

def esbpl(energy):
    eng = energy
    return eng * sbpl(eng)

In [9]:
Flux_Ph = integrate.quad(sbpl, emin, emax, limit=100)[0]
Flux_En = integrate.quad(esbpl, emin, emax, limit=100)[0] * keVtoerg

print(
'''
Photon Flux:  %.9f \t photons s^-1 cm^-2
Energy Flux:  %.9e \t ergs s^-1 cm^-2
'''%(Flux_Ph, Flux_En))


Photon Flux:  6.206307228 	 photons s^-1 cm^-2
Energy Flux:  2.003884465e-06 	 ergs s^-1 cm^-2



## FUNCTION WITHOUT BEING INTEGRATED.
### WITHOUT COSH

In [10]:
def sbpl(energy):
    from numpy import log, log10, exp
    alpha, beta, ebreak, norm = get_parVals()
    eng  = energy
    k    = float(ebreak)
    d    = 0.3              
    N    = float(norm)  
    p1   = (beta - alpha)/2.
    p2   = (alpha + beta)/2.
    p3   = ( log10(100.0/k)/d )
    p4   = ( log10(eng/k)/d )
    return N * ((eng/100.0)**p2) * (10**((p1 * d * log((exp(p4) + exp(-p4))/2.)) - (p1 * d * log((exp(p3) + exp(-p3))/2.))))

def esbpl(energy):
    eng = energy
    return eng * sbpl(eng)

In [11]:
FLUX_PH = integrate.quad(sbpl, emin, emax, limit=100)[0]
FLUX_EN = integrate.quad(esbpl, emin, emax, limit=100)[0] * keVtoerg

print(
'''
Photon Flux:  %.9f \t photons s^-1 cm^-2
Energy Flux:  %.9e \t ergs s^-1 cm^-2
'''%(FLUX_PH, FLUX_EN))


Photon Flux:  6.206307228 	 photons s^-1 cm^-2
Energy Flux:  2.003884465e-06 	 ergs s^-1 cm^-2

